# 加法进位实验


<img src="https://github.com/JerrikEph/jerrikeph.github.io/raw/master/Learn2Carry.png" width=650>

In [12]:
!pip install tqdm

     -------------------------------------- 78.5/78.5 kB 875.6 kB/s eta 0:00:00


In [47]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets
import os,sys,tqdm


## 数据生成
我们随机在 `start->end`之间采样除整数对`(num1, num2)`，计算结果`num1+num2`作为监督信号。

* 首先将数字转换成数字位列表 `convertNum2Digits`
* 将数字位列表反向
* 将数字位列表填充到同样的长度 `pad2len`


In [48]:
def gen_data_batch(batch_size, start, end):
    '''在(start, end)区间采样生成一个batch的整型的数据
    Args :
        batch_size: batch_size
        start: 开始数值
        end: 结束数值
    '''
    numbers_1 = np.random.randint(start, end, batch_size)
    numbers_2 = np.random.randint(start, end, batch_size)
    results = numbers_1 + numbers_2
    return numbers_1, numbers_2, results

def convertNum2Digits(Num):
    '''将一个整数转换成一个数字位的列表,例如 133412 ==> [1, 3, 3, 4, 1, 2]
    '''
    strNum = str(Num)
    chNums = list(strNum)
    digitNums = [int(o) for o in strNum]
    return digitNums

def convertDigits2Num(Digits):
    '''将数字位列表反向， 例如 [1, 3, 3, 4, 1, 2] ==> [2, 1, 4, 3, 3, 1]
    '''
    digitStrs = [str(o) for o in Digits]
    numStr = ''.join(digitStrs)
    Num = int(numStr)
    return Num

def pad2len(lst, length, pad=0):
    '''将一个列表用`pad`填充到`length`的长度 例如 pad2len([1, 3, 2, 3], 6, pad=0) ==> [1, 3, 2, 3, 0, 0]
    '''
    lst+=[pad]*(length - len(lst))
    return lst

def results_converter(res_lst):
    '''将预测好的数字位列表批量转换成为原始整数
    Args:
        res_lst: shape(b_sz, len(digits))
    '''
    res = [reversed(digits) for digits in res_lst]
    return [convertDigits2Num(digits) for digits in res]

def prepare_batch(Nums1, Nums2, results, maxlen):
    '''准备一个batch的数据，将数值转换成反转的数位列表并且填充到固定长度
    Args:
        Nums1: shape(batch_size,)
        Nums2: shape(batch_size,)
        results: shape(batch_size,)
        maxlen:  type(int)
    Returns:
        Nums1: shape(batch_size, maxlen)
        Nums2: shape(batch_size, maxlen)
        results: shape(batch_size, maxlen)
    '''
    Nums1 = [convertNum2Digits(o) for o in Nums1]
    Nums2 = [convertNum2Digits(o) for o in Nums2]
    results = [convertNum2Digits(o) for o in results]
    
    Nums1 = [list(reversed(o)) for o in Nums1]
    Nums2 = [list(reversed(o)) for o in Nums2]
    results = [list(reversed(o)) for o in results]
    
    Nums1 = [pad2len(o, maxlen) for o in Nums1]
    Nums2 = [pad2len(o, maxlen) for o in Nums2]
    results = [pad2len(o, maxlen) for o in results]
    
    return Nums1, Nums2, results

# 建模过程， 按照图示完成建模

In [49]:
class myRNNModel(keras.Model):
    def __init__(self):
        super(myRNNModel, self).__init__()
        self.embed_layer = tf.keras.layers.Embedding(10, 32, 
                                                    batch_input_shape=[None, None])
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(64)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(10)
        
#     @tf.function
    def call(self, num1, num2):
        '''
        此处完成上述图中模型
        '''
        emb1 = self.embed_layer(num1) 
        emb2 = self.embed_layer(num2) 
        out_rnn = self.rnn_layer(tf.concat([emb1, emb2], axis=-1))
        logits = self.dense(out_rnn)
        return logits

In [50]:
# @tf.function
def compute_loss(logits, labels):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    return tf.reduce_mean(losses)

# @tf.function
def train_one_step(model, optimizer, x, y, label):
    with tf.GradientTape() as tape:
        logits = model(x, y)
        loss = compute_loss(logits, label)

    # compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# def train(steps, model, optimizer):
#     loss = 0.0
#     accuracy = 0.0
#     for step in range(steps):
#         datas = gen_data_batch(batch_size=200, start=0, end=555555555)
#         Nums1, Nums2, results = prepare_batch(*datas, maxlen=11)
    
#         # 假设 Nums1, Nums2 和 results 是 Python 列表
#         Nums1_tensor = tf.convert_to_tensor(Nums1, dtype=tf.int32)  # 转换为 TensorFlow 张量
#         Nums2_tensor = tf.convert_to_tensor(Nums2, dtype=tf.int32)  # 转换为 TensorFlow 张量
#         results_tensor = tf.convert_to_tensor(results, dtype=tf.int32)  # 转换为 TensorFlow 张量

#         # 现在可以将这些张量传递给 train_one_step 函数
#         loss = train_one_step(model, optimizer, Nums1_tensor, Nums2_tensor, results_tensor)

# #         loss = train_one_step(model, optimizer, tf.constant(Nums1, dtype=tf.int32), 
# #                               tf.constant(Nums2, dtype=tf.int32),
# #                               tf.constant(results, dtype=tf.int32))
#         if step%50 == 0:
# #             print('step', step, ': loss', loss.numpy())
#             # 使用 TensorFlow 的打印函数
#             tf.print('step', step, ': loss', loss)

#     return loss

def train(steps, model, optimizer):
    loss = 0.0
    accuracy = 0.0
    for step in range(steps):
        datas = gen_data_batch(batch_size=200, start=0, end=555555555)
        Nums1, Nums2, results = prepare_batch(*datas, maxlen=11)
    
        # 假设 Nums1, Nums2 和 results 是 Python 列表
        Nums1_tensor = tf.convert_to_tensor(Nums1, dtype=tf.int32)  # 转换为 TensorFlow 张量
        Nums2_tensor = tf.convert_to_tensor(Nums2, dtype=tf.int32)  # 转换为 TensorFlow 张量
        results_tensor = tf.convert_to_tensor(results, dtype=tf.int32)  # 转换为 TensorFlow 张量

        # 现在可以将这些张量传递给 train_one_step 函数
        loss = train_one_step(model, optimizer, Nums1_tensor, Nums2_tensor, results_tensor)

        # 使用 TensorFlow 的打印函数来打印损失
        if step % 50 == 0:
            tf.print('step', step, ': loss', loss)

    return loss


def evaluate(model):
    datas = gen_data_batch(batch_size=2000, start=555555555, end=999999999)
    Nums1, Nums2, results = prepare_batch(*datas, maxlen=11)
    logits = model(tf.constant(Nums1, dtype=tf.int32), tf.constant(Nums2, dtype=tf.int32))
    logits = logits.numpy()
    pred = np.argmax(logits, axis=-1)
    res = results_converter(pred)
    for o in list(zip(datas[2], res))[:20]:
        print(o[0], o[1], o[0]==o[1])

    print('accuracy is: %g' % np.mean([o[0]==o[1] for o in zip(datas[2], res)]))


In [51]:
optimizer = optimizers.Adam(0.001)
model = myRNNModel()

In [52]:
train(3000, model, optimizer)
evaluate(model)

step 0 : loss 2.30033
step 50 : loss 1.91028404
step 100 : loss 1.90698886
step 150 : loss 1.89869142
step 200 : loss 1.88984489
step 250 : loss 1.8932873
step 300 : loss 1.88818443
step 350 : loss 1.88368762
step 400 : loss 1.88414478
step 450 : loss 1.87054884
step 500 : loss 1.87755084
step 550 : loss 1.8829608
step 600 : loss 1.87106109
step 650 : loss 1.87245297
step 700 : loss 1.88969421
step 750 : loss 1.88812411
step 800 : loss 1.88011074
step 850 : loss 1.8658576
step 900 : loss 1.87331116
step 950 : loss 1.86360264
step 1000 : loss 1.86363864
step 1050 : loss 1.82845283
step 1100 : loss 1.7803458
step 1150 : loss 1.68835318
step 1200 : loss 1.54750252
step 1250 : loss 1.34005
step 1300 : loss 1.13863945
step 1350 : loss 0.971855223
step 1400 : loss 0.836984336
step 1450 : loss 0.735088766
step 1500 : loss 0.648647666
step 1550 : loss 0.559552729
step 1600 : loss 0.502735257
step 1650 : loss 0.436285466
step 1700 : loss 0.372326165
step 1750 : loss 0.326484
step 1800 : loss 0.